<a href="https://colab.research.google.com/github/guysa-AMG/meta-rag/blob/main/Metallm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>South African carbon Emission

In [ ]:
!pip install -r requirements.txt

In [2]:
import os
from getpass import getpass
import nest_asyncio
from llama_index.core import Settings
from llama_index.llms.openrouter import OpenRouter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.llama_pack import download_llama_pack
from google.colab import userdata
import requests
import re

nest_asyncio.apply()

In [4]:
os.environ["OPEN_ROUTER_API"] = userdata.get("OPEN_ROUTER_API")
if os.environ["OPEN_ROUTER_API"] is None:
  os.environ["OPEN_ROUTER_API"] = getpass("Enter your OpenRouter API key: ")


In [ ]:
llm = OpenRouter(
    api_key=os.environ["OPEN_ROUTER_API"],
    model="meta-llama/llama-3.3-70b-instruct:free",
    max_tokens=512,
    temperature=0.1,
    timeout=60,
    system_prompt=(
        "You are an expert RAG system that answers ONLY using the provided context. "
        "Never hallucinate. Never guess. If the answer is not in the context, say so. "
        "Provide short, clear, factual responses with 2–4 evidence bullets."
    ),
)
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = llm
Settings.embed_model = embed_model


In [6]:
from utils import download_pdf_from_drive


In [ ]:
file_id ="1gxGBfglAi93cnw5IAtPPNB4p8oSZqMOd"

DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)
pdf_path = os.path.join(DATA_DIR, "source.pdf")

download_pdf_from_drive(file_id,pdf_path)


In [ ]:
documents = SimpleDirectoryReader(input_files=[pdf_path]).load_data()
print(f"[!] Loaded {len(documents)} Pages.")

In [ ]:
semantic_embed = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
parser = SemanticSplitterNodeParser( buffer_size=3,breakpoint_percentile_threshold=95, embed_model=semantic_embed)
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
for n in nodes:
    n.metadata["source"]     = pdf_path
    n.metadata["chunk_type"] = "semantic"

print(f"[!] Created {len(nodes)} semantic nodes.")

In [ ]:
from llama_index.core.llama_pack import download_llama_pack

QueryRewritingRetrieverPack = download_llama_pack(
    "QueryRewritingRetrieverPack",
    "./query_rewriting_pack",
)

query_rewriting_pack = QueryRewritingRetrieverPack(
    nodes,
    chunk_size=256,
    vector_similarity_top_k=8,
    fusion_similarity_top_k=8,
    num_queries=6,
)

print("AQF RAG Engine Ready!")

In [12]:
def safe_rag_run(question, retries=3):
    for attempt in range(retries):
        try:
            resp = query_rewriting_pack.run(question)

            if resp is None or str(resp).strip() == "":
                raise ValueError("Empty LLM response.")
            return resp

        except Exception as e:
            print(f"Error: {e}")
            print(f"Retrying ({attempt+1}/{retries})...")

    return "[x] Could not generate a valid answer after retries."

In [ ]:
while True:
    user_question = input("What would you like to know?: ").strip()
    if user_question.lower() == "q":
        break
    response = safe_rag_run(user_question)
    print("\n──────────────────────────────────────────────")
    print("? QUESTION:")
    print(user_question)
    print("\n# ANSWER:")
    print(response)
    print("──────────────────────────────────────────────\n")